In [ ]:
# package import
import chipwhisperer as cw
from cw_plugins.scopes import *
from cw_plugins.targets import *
from tqdm import tqdm
from ipywidgets import Dropdown
import pyvisa

### VISA Instrument Selector
Select a VISA instrument of an oscilloscope you want to use.

In [ ]:
VisaAddress = None
inst_sel = Dropdown(options=pyvisa.ResourceManager().list_resources(), description='Select VISA Instrument:')
inst_sel.options = list(inst_sel.options) + ['None']
def set_visa_address(change):
    global VisaAddress
    VisaAddress = change['new']
inst_sel.observe(set_visa_address, names='value')
display(inst_sel)

### Connect to the oscilloscope and channel settings
At least, you need to configure trigger settings and trace settings by the following methods.

`config_trigger_channel(mode, channel, scale, offset)`
* `mode`: Trigger mode.
  * TriggerMode.EDGE_RISE: trigger with rising edge.
  * TriggerMode.EDGE_FALL: trigger with falling edge.
  * TriggerMode.EDGE_ANY: trigger with both rising and falling edges.
* `channel`: channel number
* `scale`: vertical scale (V/div)
* `offset`: vertical offset (V)

`config_trace_channel(channel, scale, offset, period, delay = 0, ....)`
* `channel`: channel number
* `scale`: vertical scale (V/div)
* `offset`: vertical offset (V)
* `period`: acquired waveform period (s)
* `delay`: delay time to start acquiring waveform after trigger (s)
* `...`: other device-specific parameters


In [ ]:
# connect to the oscilloscope
scope = Oscilloscope(VisaAddress)
# setting the scope
# channel 3 is the trigger channel
scope.config_trigger_channel(TriggerMode.EDGE_FALL, 3, 1, 0)
# channel 1 is the trace channel, 2mV/div, trace period 4us
scope.config_trace_channel(1, milliVolt(2), -1, microSecond(4), delay = 0, impedance=50)


### Target device
This plugin provides additional target devices as follows:
* SakuraX: Sakura-X board configured with its reference design.  
Options:
  * serial_port (str): path to the serial port connected to the Sakura-X board.
* ESP32: ESP32 board
  * serial_port (str): path to the serial port connected to the ESP32 board.
  * baudrate (int): baudrate of the serial port.
* SakuraXShellExampleAES128BitRTL, SakuraXShellExampleAES128BitHLS
Sample of AES128 implementation using Sakura-X Shell.

In [ ]:
# connect to the target
target = cw.target(scope, SakuraX, serial_port = "/dev/ttyUSB0")

### Create project
Create a project with the following code.

In [ ]:
### create chipwhisperer project
project = cw.create_project("cpa_sakura", overwrite=True)

### Start trace acquisition
Start trace acquisition by the following code.

In [ ]:
# acquire traces
ktp = cw.ktp.Basic()
# generate a key
key = ktp.next_key()
print("key: ", key)

trace_count = 100

progress_bar = tqdm(total=trace_count, desc="Capturing traces")
while len(project.traces) < trace_count:
    text = ktp.next_text()
    trace = cw.capture_trace(scope, target, text, key)
    if trace is None:
        continue
    project.traces.append(trace)
    progress_bar.update(1)

### Save project
Save the project by the following code.

In [ ]:
# save the project
project.save()